In [ ]:
names = ['thorax', 'neck', 'a_L1', 'a_L2', 'a_R1', 'a_R2', 'leg_f_L1', 'leg_f_L2', 'leg_f_R1', 'leg_f_R2', 'leg_m_L1', 'leg_m_L2', 'leg_m_R1', 'leg_m_R2', 'a_R0', 'a_L0', 'leg_f_R0', 'leg_f_L0', 'leg_m_R0', 'leg_m_L0', 'eye_L', 'eye_R', 'm_L0', 'm_R0', 'm_L1', 'm_R1']

In [2]:
from src.collision import CollisionDetector
obj_folder = "./segmentation/28"
t = CollisionDetector(obj_folder=obj_folder, skeleton_toml_path="skeleton.toml", pose_csv="./tracking/28/240905-1616_session28_track99_points3d.csv")

PosixPath('segmentation/28/240905-1616_seed_session28.toml') generated an exception: File segmentation/28/240905-1616_seed_session28.toml ignored, not .dae or .obj file


In [3]:
t = CollisionDetector(obj_folder=obj_folder, skeleton_toml_path="skeleton.toml", pose_csv="./tracking/28/240905-1616_session28_track99_points3d.csv", obj_ref_frame=2599, obj_transform_toml=f"{obj_folder}/240905-1616_seed_session28.toml")

In [4]:
from src.visual_widget import MultiViewer
V = MultiViewer([t.animal], [t.obj], 563)

2025-03-06 09:26:04.897 python[45407:8599855] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-06 09:26:04.931 python[45407:8599855] +[IMKInputSession subclass]: chose IMKInputSession_Modern
ERROR:src.animal:No rays present for animal at frame 843
ERROR:src.animal:No rays present for animal at frame 845
ERROR:src.animal:No rays present for animal at frame 846
ERROR:src.animal:No rays present for animal at frame 847
ERROR:src.animal:No rays present for animal at frame 854
ERROR:src.animal:No rays present for animal at frame 855
ERROR:src.animal:No rays present for animal at frame 857


In [ ]:
V.scene_widget.scene.graph.show()